In [ ]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
%matplotlib inline
x = np.random.normal(0.0, 0.55, (10000, 1))
y = x * 0.1 + 0.3 + np.random.normal(0.0, 0.03, (10000,1))
                     
plt.plot(x, y, 'r.')
plt.show()

X = tf.placeholder(tf.float32, shape=[None, 1])
W = tf.Variable(tf.random_normal([1]))
b = tf.Variable(tf.zeros([1]))

h = X*W+b

Y = tf.placeholder(tf.float32, shape = [None, 1])
Loss = tf.reduce_mean(tf.square(h - Y))
optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(Loss)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Training loop
for epoch in range(500):
    _, t_loss = sess.run([optimizer, Loss], feed_dict={X:x, Y:y})
    
    print("Epoch : ", epoch, " Loss : ", t_loss)
    
    if epoch ==0 :
        y_pred = sess.run(h, feed_dict={X:x})
        plt.plot(x, y, 'r.')
        plt.plot(x, y_pred, 'b.')
        plt.show()
    elif (epoch+1) % 100 == 0 :
        y_pred = sess.run(h, feed_dict={X:x})
        plt.plot(x, y, 'r.')
        plt.plot(x, y_pred, 'b.')
        plt.show()

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import SGD, Adam

x = np.random.normal(0.0, 0.55, (10000, 1))
y = x * 0.1 + 0.3 + np.random.normal(0.0, 0.03, (10000,1))
                     
plt.plot(x, y, 'r.')
plt.show()

_input_shape=Input(shape=(1,))
h = Dense(1)(_input_shape)

model = Model(_input_shape, h)
model.compile(optimizer=SGD(lr=0.05), loss='mse')

history = model.fit(x, y, epochs=500, batch_size=10000)

pred = model.predict(x)
plt.plot(x, y, 'r.')
plt.plot(x, pred, 'b.')
plt.show()

In [ ]:
import numpy as np
from collections import deque
from matplotlib import pyplot as plt
from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import SGD, Adam

import gym

env = gym.make('CartPole-v0')

# Constants defining our neural network
learning_rate = 1e-1
input_size = env.observation_space.shape[0]
output_size = env.action_space.n


X = Input(shape=(input_size,))
h = Dense(input_size)(X)
Qpred = Dense(output_size)(h)

model =Model(X,Qpred)
model.summary()

In [ ]:
model.compile(optimizer = Adam(lr = learning_rate), loss ='mse')

max_episodes = 1000
dis = 0.9
step_history = []

for episode in range(max_episodes):
    e = 1. / ((episode / 10) + 1)
    step_count = 0
    state = env.reset()
    done = False

    # The Q-Network training
    while not done:
        step_count += 1
        x = np.reshape(state, [1, input_size])
        # Choose an action by greedily (with e chance of random action) from
        # the Q-network
        Q = model.predict(x)
        
        if np.random.rand(1) < e:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q)

        # Get new state and reward from environment
        next_state, reward, done, _ = env.step(action)
        if done:
            Q[0, action] = -100
        else:
            x_next = np.reshape(next_state, [1, input_size])
            # Obtain the Q' values by feeding the new state through our network
            Q_next =model.predict(x_next)
            Q[0, action] = reward + dis * np.max(Q_next)

        # Train our network using target and predicted Q values on each episode
        model.fit(x,Q, verbose = 0)
        state = next_state

    step_history.append(step_count)
    print("Episode: {}  steps: {}".format(episode, step_count))
    # If last 10's avg steps are 500, it's good enough
    if len(step_history) > 10 and np.mean(step_history[-10:]) > 500:
        break

# See our trained network in action
observation = env.reset()
reward_sum = 0
while True:
    env.render()

    x = np.reshape(observation, [1, input_size])
    Q = model.predict(x)
    action = np.argmax(Q)

    observation, reward, done, _ = env.step(action)
    reward_sum += reward
    if done:
        print("Total score: {}".format(reward_sum))
        break

In [ ]:
plt.plot(step_history)